In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import time
import random
import datetime
import os

In [ ]:
class Scrape():
    def __init__(self,wait=3,max=None):
        self.response = None
        self.df = pd.DataFrame()
        self.wait = wait
        self.max = max
        self.timeout = 5

    def request(self,url,encoding='utf-8',wait=None,max=None,console=True):
        '''
        指定したURLからページを取得する。
        取得後にwaitで指定された秒数だけ待機する。
        max が指定された場合、waitが最小値、maxが最大値の間でランダムに待機する。

        Params
        ---------------------
        url:str
            URL
        encoding:str
            ページのエンコード
        wait:int
            ウェイト秒
        max:int
            ウェイト秒の最大値
        console:bool
            状況をコンソール出力するか
        Returns
        ---------------------
        soup:BeautifulSoupの戻り値
        '''
        self.wait = self.wait if wait is None else wait
        self.max = self.max if max is None else max

        start = time.time()
        response = requests.get(url,timeout = self.timeout)
        response.encoding = encoding
        time.sleep(random.randint(self.wait,self.wait if self.max is None else self.max))

        if console:
            tm = datetime.datetime.now().strftime('%Y/%m/%d %H:%M:%S')
            lap = time.time() - start
            print(f'{tm} : {url}  経過時間 : {lap:.3f} 秒')

        return BeautifulSoup(response.text, "html.parser")

    def get_href(self,soup,contains = None):
        '''
        soupの中からアンカータグを検索し、空でないurlをリストで返す
        containsが指定された場合、更にその文字列が含まれるurlだけを返す

        Params
        ---------------------
        soup:str
            BeautifulSoupの戻り値
        contains:str
            抽出条件となる文字列

        Returns
        ---------------------
        return :[str]
            条件を満たすurlのリスト
        '''
        urls = list(set([url.get('href') for url in soup.find_all('a')]))
        if contains is not None:
           return [url for url in urls if self.contains(url,contains)]
        return [url for url in urls if urls is not None or urls.strip() != '']

    def get_src(self,soup,contains = None):
        '''
        soupの中からimgタグを検索し、空でないsrcをリストで返す
        containsが指定された場合、更にその文字列が含まれるurlだけを返す

        Params
        ---------------------
        soup:str
            BeautifulSoupの戻り値
        contains:str
            抽出条件となる文字列

        Returns
        ---------------------
        return :[str]
            条件を満たすurlのリスト
        '''
        urls = list(set([url.get('src') for url in soup.find_all('img')]))
        if contains is not None:
           return [url for url in urls if contains(url,self.contains)]
        return [url for url in urls if urls is not None or urls.strip() != '']

    def contains(self,line,kwd):
        '''
        line に kwd が含まれているかチェックする。
        line が None か '' の場合、或いは kwd が None 又は '' の場合は Trueを返す。

        Params
        ---------------------
        line:str
            HTMLの文字列
        contains:str
            抽出条件となる文字列

        Returns
        ---------------------
        return :[str]
            条件を満たすurlのリスト
        '''
        if line is None or line.strip() == '':
            return False
        if kwd is None or kwd == '':
            return True
        return kwd in line 

    def omit_char(self,values,omits):
        '''
        リストで指定した文字、又は文字列を削除する

        Params
        ---------------------
        values:str
            対象文字列
        omits:str
            削除したい文字、又は文字列

        Returns
        ---------------------
        return :str
            不要な文字を削除した文字列
        '''
        for n in range(len(values)):
            for omit in omits:
                values[n] = values[n].replace(omit,'')
        return values

    def add_df(self,values,columns,omits = None):
        '''
        指定した値を　DataFrame に行として追加する
        omits に削除したい文字列をリストで指定可能

        Params
        ---------------------
        values:[str]
            列名
        omits:[str]
            削除したい文字、又は文字列
        '''
        if omits is not None:
            values = self.omit_char(values,omits)
            columns = self.omit_char(columns,omits)

        df = pd.DataFrame(values,index=self.rename_column(columns))
        self.df = pd.concat([self.df,df.T],ignore_index=True)

    def to_csv(self,filename,dropcolumns=None):
        '''
        DataFrame をCSVとして出力する
        dropcolumns に削除したい列をリストで指定可能

        Params
        ---------------------
        filename:str
            ファイル名
        dropcolumns:[str]
            削除したい列名
        '''
        if dropcolumns is not None:
            self.df.drop(dropcolumns,axis=1,inplace=True)
        self.df.insert(0,'id',self.df.index)
        self.df.to_csv(filename,index=False)

    def get_text(self,soup):
        '''
        渡された soup が Noneでなければ textプロパティの値を返す

        Params
        ---------------------
        soup: bs4.element.Tag
            bs4でfindした結果の戻り値

        Returns
        ---------------------
        return :str
            textプロパティに格納されている文字列
        '''

        return ' ' if soup == None else soup.text

    def rename_column(self,columns):
        '''
        重複するカラム名の末尾に連番を付与し、ユニークなカラム名にする
            例 ['A','B','B',B'] → ['A','B','B_1','B_2']

        Params
        ---------------------
        columns: [str]
            カラム名のリスト
          
        Returns
        ---------------------
        return :str
            重複するカラム名の末尾に連番が付与されたリスト
        '''
        lst = list(set(columns))
        for column in columns:
            dupl = columns.count(column)
            if dupl > 1:
                cnt = 0
                for n in range(0,len(columns)):
                    if columns[n] == column:
                        if cnt > 0:
                            columns[n] = f'{column}_{cnt}'
                        cnt += 1
        return columns

In [ ]:
def scrape_kakaku(url):
    scr = Scrape(wait=3,max=5)

    #レビューのURLからCategoryCD=2110/までを取り出す
    url2 = url[:url.find('?lid')]

    #製品画像のダウンロードディレクトリ作成
    imgdir = "./image"
    if not os.path.exists(imgdir):
        os.mkdir(imgdir)

    for n in range(1,316):
        #リクエストするURLを設定
        if n == 1:
            target = url
        else:
            #商品の指定ページのURLを生成
            target = url2+f'PageNo={n}/'

        #レビューページの取得
        soup = scr.request(target,encoding='shift_jis')
        #ページ内の製品カテゴリを一括取得
        categories = soup.find_all('table',class_='catebox')
        #ページ内のレビュー記事を一括取得
        reviews = soup.find_all('div',class_='revMainClmWrap')
        #ページ内のすべてと評価を一括取得
        evals = soup.find_all('div',class_='reviewBoxWtInner')

        print(f'レビュー数:{len(reviews)}')

        #ページ内の全てのレビューをループで取り出す
        for category,review,eval in zip(categories,reviews,evals):
            #製品画像を取得
            imgurl = scr.get_src(category.find('td',class_='prdimg'))[0]
            imgurl = imgurl.replace("/m/", "/l/")    # 大きい画像にする
            img = imgdir + "/" + os.path.basename(imgurl)
            prdimg = "Noimage.jpg"    #NoImageの画像
            r = requests.get(imgurl)
            if r.status_code == 200:
                with open(img, "wb") as f:
                    f.write(r.content)
                    prdimg = os.path.basename(imgurl)
            time.sleep(1)
            #製品名を取得
            prdctgry = scr.get_text(category.find('td',class_='prdctgry')).split('>')
            prdmaker = prdctgry[1].strip()
            prdname = prdctgry[2 if (len(prdctgry) < 4) else 3].strip()
            #レビューのタイトルを取得
            title = scr.get_text(review.find('div',class_='reviewTitle'))
            #レビューの内容を取得
            comment = scr.get_text(review.find('p',class_='revEntryCont')).replace('<br>','')

            #満足度（デザイン、使いやすさ、洗浄力、静音性、サイズ、機能・メニュー、・・・・・の値を取得
            tables = eval.find_all('table')
            star = scr.get_text(tables[0].find('td'))
            date = scr.get_text(eval.find('p',class_='entryDate clearfix'))
            date = date[:date.find('日')+1]
            ths = tables[1].find_all('th')
            tds = tables[1].find_all('td')

            columns = ['prdimg','prdmaker','prdname','title','star','date','comment']
            values = [prdimg,prdmaker,prdname,title,star,date,comment]

            for th,td in zip(ths,tds):
                columns.append(th.text)
                values.append(td.text)

            #レビューの詳細データを取得
            detail = review.find('div',class_='revDetailData')
            if detail is not None:
                dl = detail.find('dl',class_='clearfix')
                dts = dl.find_all('dt')
                dds = dl.find_all('dd')
                detail = ''
                for dt, dd in zip(dts, dds):
                    detail += ' ' + dt.text + ':' + dd.text
                for dd in dds[len(dts):]:
                    detail += '|' + dd.text
                detail = detail.strip()
            else:
                detail = ''
            columns.append('detail')
            values.append(detail)

            #DataFrameに登録
            scr.add_df(values,columns,['<br>'])

        #ページ内のレビュー数が15未満なら、最後のページと判断してループを抜ける
        if len(reviews) < 15:
            break

    #スクレイプ結果をCSVに出力
    scr.to_csv(" ") #保存ディレクトリ指定

In [ ]:
scrape_kakaku('') #URL指定